In [ ]:
#!pip install meetup.api

In [14]:
import meetup.api
import json 
import requests 
import time 
import codecs
import sys
import io
import re
import pandas as pd
import sqlalchemy as sa

In [2]:
def superFetch(groupname, key):
    print("Start fetching...")
    client = meetup.api.Client(key)
    print('request status:',end=' ')
    group = client.GetGroup({'urlname':groupname})
    gid = group.id
    total = group.members
    want = 'id,name,state,city,lon,lat'
    limit = 200
    request = total//limit+1
    offsetValue = 0
    name = []
    uid = []
    state = []
    city = []
    lon = []
    lat = []
    for i in range(request):
        print('progress: ',offsetValue+1,'/',request, sep='')
        print('request status:',end=' ')
        try:
            members = client.GetMembers(group_id=gid, only=want,page=limit,offset=offsetValue)
            offsetValue+=1
            rList= members.results
            for j in rList:
                if len(j.keys())==4:
                    name.append(j['name'])
                    uid.append(j['id'])
                    state.append(j['state'])
                    city.append(j['city'])
                else:
                    uid.append(j['id'])
                    try:
                        name.append(j['name'])
                    except KeyError:
                        name.append('null')
                    try:
                        state.append(j['state'])
                    except KeyError:
                        state.append('null')
                    try:
                        city.append(j['city'])
                    except:
                        city.append('null')
                    try:
                        lon.append(j['lon'])
                        lat.append(j['lat'])
                    except:
                        lon.append('null')
                        lat.append('null')
        except json.decoder.JSONDecodeError:
            print("JSONDecodeError")
    print("fetching complete")
    df = pd.DataFrame({"name":name, "uid":uid, "state":state, "city":city, "lon":lon, "lat":lat})
    return df


key = '1a60703a102a6252424875415c1b21'
df = superFetch('DC-Tech-Meetup', key)

Start fetching...
request status: 29/30 (10 seconds remaining)
progress: 1/125
request status: 28/30 (10 seconds remaining)
progress: 2/125
request status: 27/30 (5 seconds remaining)
progress: 3/125
request status: 

KeyboardInterrupt: 

In [ ]:
df["state"] = df["state"].replace("DC", 1)
df["state"] = df["state"].replace("MD", 2)
df["state"] = df["state"].replace("VA", 3)

for index,row in df.iterrows():
    if df.loc[index, "state"] != 1 and df.loc[index, "state"] != 2 and df.loc[index, "state"] != 3:
        df.loc[index, "state"] = 4

In [ ]:
print(df)

In [ ]:
def scrapeData(protocol, host, resource):
    '''
    This function takes elements of a URL and returns the raw text from
    the webpage.
    Parameters: protocol, host, resource
    Returns: response text
    '''
    template = "{}://{}{}"
    url = template.format(protocol, host, resource)
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.text
    print("Error")

txt = scrapeData("https", "en.wikipedia.org", "/wiki/Washington_metropolitan_area")

In [24]:
def extractSectionBetween(txt, start, end):
    '''
    Slices text by substring boundaries rather than index boundaries.
    Parameters: text to slice, start string, end string
    Returns: trimmed string
    '''
    startInd = txt.find(start)
    endInd = txt.find(end)
    return txt[startInd:endInd]

txt = extractSectionBetween(txt, "92.3 years", "The Washington, D.C. area has the largest science")

In [25]:
def recreateTable(txt):
    '''
    Takes text containing a table in raw HTML format and
    places desired data in a pandas data frame.
    Parameters: html text
    Returns: pandas data frame
    '''
    pat = re.compile(r"<td><a .*?>(?P<county>.*?)<\/a>\n<\/td>\n<td .*?>(?P<pop>.*?)\n<\/td>")
    countyid = []
    i = 1
    county = []
    state = []
    pop = []
    for m in pat.finditer(txt):
        county.append(m["county"].split(",")[0].strip())
        state.append(m["county"].split(",")[1].strip())
        pop.append(int(m["pop"].replace(",", "")))
        countyid.append(i)
        i += 1
    return pd.DataFrame({"CountyID":countyid, "County":county, "State":state, "Population":pop})
    
popTable = recreateTable(txt)

popTable["State"] = popTable["State"].replace("D.C.", 1)
popTable["State"] = popTable["State"].replace("Maryland", 2)
popTable["State"] = popTable["State"].replace("Virginia", 3)



print(popTable)
    


    CountyID                  County  State  Population
0          1              Washington      1      681170
1          2          Calvert County      2       91251
2          3          Charles County      2      157705
3          4        Frederick County      2      247591
4          5       Montgomery County      2     1043863
5          6  Prince George's County      2      908049
6          7              Alexandria      3      155810
7          8        Arlington County      3      230050
8          9           Clarke County      3       14374
9         10         Culpeper County      3       50083
10        11          Fairfax County      3     1138652
11        12            Fairfax City      3       24164
12        13            Falls Church      3       14014
13        14         Fauquier County      3       69069
14        15          Fredericksburg      3       28297
15        16          Loudoun County      3      385945
16        17                Manassas      3     

In [3]:
def getCreds(filename,subset,defaults={}):
    '''
    This function helps us connect to a database on hadoop2.  The
    userid password information is stored in a local file encoded
    in json format.  
    Parameters:
    filename is where json encoded userid/password information stored
    subset: dictionary key in json file
    defaults: alternative way to connect to database
    Returns: subdictionary based on subset parameter.
    '''
    try:
        with open(filename,'r') as file:
            D = json.load(file)
            file.close()
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults

def db_setup(userid,password):
    '''
    This function connects us to a database using our login credentials. 
    Paramters:
    userid - string with our userid
    password - string with our password
    database - string with database name
    Returns:
    SQL engine and SQL connection objects
    '''
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/'
    cstring = template.format(userid,password)
    engine = sa.create_engine(cstring)
    connection = engine.connect()
    
    return engine, connection

# creds = getCreds('creds.json','mysql',defaults = {'user':'studen_j1', 'password':'studen_j1'})
creds = {'user': 'brown_b1', 'password': 'brown_b1'}

In [4]:
try:
    connection.close()
    del engine
except:
    pass
engine, connection = db_setup(creds['user'],creds['password'])

In [44]:
makeDB = "CREATE DATABASE brown_b1;"
#connection.execute("commit")
#connection.execute(makeDB)

In [7]:
def db_setup(userid,password, database):
    '''
    This function connects us to a database using our login credentials. 
    Paramters:
    userid - string with our userid
    password - string with our password
    database - string with database name
    Returns:
    SQL engine and SQL connection objects
    '''
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(userid,password,database)
    engine = sa.create_engine(cstring)
    connection = engine.connect()
    
    return engine, connection

try:
    connection.close()
    del engine
except:
    pass
engine, connection = db_setup(creds['user'],creds['password'], "brown_b1")

Exception during reset or similar
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/sqlalchemy/pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "/anaconda3/lib/python3.6/site-packages/sqlalchemy/pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "/anaconda3/lib/python3.6/site-packages/sqlalchemy/dialects/mysql/base.py", line 1796, in do_rollback
    dbapi_connection.rollback()
  File "/anaconda3/lib/python3.6/site-packages/mysql/connector/connection_cext.py", line 332, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: Commands out of sync; you can't run this command now


In [49]:


makeCountyPopulations = "CREATE TABLE CountyPopulations(\
CountyID INT NOT NULL,\
CountyName VARCHAR(30),\
StateID INT,\
Population INT,\
PRIMARY KEY(CountyID)\
);"

makeStates = "CREATE TABLE States(\
StateID INT NOT NULL,\
State VARCHAR(30),\
PRIMARY KEY(StateID)\
);"

makeUsers = "CREATE TABLE Users(\
MemberID INT NOT NULL,\
Nickname VARCHAR(30),\
StateID INT,\
UserCity VARCHAR(30),\
UserLongitude DECIMAL(5, 2),\
UserLatitude DECIMAL(5, 2),\
PRIMARY KEY(MemberID)\
);"

#connection.execute(makeCountyPopulations)
#connection.execute(makeStates)
connection.execute(makeUsers)


In [50]:
countyRowTemplate = "INSERT INTO CountyPopulations\
(CountyID, CountyName, StateID, Population)\
VALUES({},{},{},{});"

stateRows = "INSERT INTO States\
(StateID, State)\
VALUES(1,'DC'),(2, 'Maryland'),(3,'Virginia'),(4,'OtherState');"

usersRowTemplate = "INSERT INTO Users\
(MemberID, Nickname, StateID, UserCity, UserLongitude, UserLatitude)\
VALUES({},{},{},{},{},{});"

#connection.execute(stateRows)


# for i in range(len(popTable)):
#     countyRow = countyRowTemplate.format(popTable.iloc[i, 0], "'"+popTable.iloc[i, 1].replace("'", "")+"'", popTable.iloc[i, 2], popTable.iloc[i, 3])
#     connection.execute(countyRow)
    

    
for i in range(len(df)):
    if i % 1000 == 0:
        print(i)
    usersRow = usersRowTemplate.format(df.iloc[i,1], "'"+df.iloc[i, 0].replace("'", "").replace("\\", "")+"'", df.iloc[i, 2], "'"+df.iloc[i, 3].replace("'", "")+"'", df.iloc[i, 4], df.iloc[i, 5])
    connection.execute(usersRow)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [48]:
len(df)

24897

In [ ]:
creds = {'user': 'brown_b1', 'password': 'brown_b1'}

try:
    connection.close()
    del engine
except:
    pass
engine, connection = db_setup(creds['user'],creds['password'], "brown_b1")

In [ ]:
def SQLtoPandas(sqlQuery):
    '''
    
    
    
    '''
    connection.execute(sqlQuery)
    queryProxy = connection.execute(sqlQuery)
    pandasDf = pd.DataFrame(queryProxy.fetchall(), columns = queryProxy.keys())
    return pandasDf

In [41]:
# dbconnect = "USE brown_b1;"

PopQuery = '''
SELECT s.State, sum(cp.Population) as StatePopulation2016\
,ROUND(SUM(cp.Population)/(SELECT SUM(cp.Population) FROM CountyPopulations AS cp)*100, 2) 
AS Percentage FROM CountyPopulations as cp
INNER JOIN States AS s ON s.StateID = cp.StateID
GROUP BY cp.StateID
ORDER BY cp.StateID;
'''

actualPopDf = SQLtoPandas(PopQuery)

actualPopDf

,State,StatePopulation2016,Percentage
0,DC,681170,11.21
1,Maryland,2448459,40.30
2,Virginia,2945980,48.49


In [43]:
UserPopQuery = '''
SELECT s.State, count(u.MemberID) as UserPop,ROUND(COUNT(u.MemberID)/(SELECT COUNT(u.MemberID) FROM Users AS u)*100, 2)
AS Percentage FROM Users AS u
INNER JOIN States AS s ON s.StateID = u.StateID
WHERE u.StateID in (1,2,3)
GROUP BY u.StateID
ORDER BY u.StateID;
'''

UserPopDf = SQLtoPandas(UserPopQuery)
print(UserPopDf)


UserPopDf.to_csv('UserPopDf.csv')

      State  UserPop Percentage
0        DC    12752      51.22
1  Maryland     3686      14.80
2  Virginia     5445      21.87


In [44]:
UserDemographicQuery = '''
SELECT u.MemberID, u.UserLongitude, u.UserLatitude, u.UserCity, s.State
FROM Users as u
INNER JOIN States as s on s.stateID = u.stateID
WHERE u.StateID in (1,2,3);
'''

UserDemographicDf = SQLtoPandas(UserDemographicQuery)
UserDemographicDf

UserDemographicDf.to_csv("UserDemographicDf.csv")